In [1]:
!pip install nltk
!pip install pandas
!pip install torch
!pip install chromadb
!pip install sentence-transformers
!pip install whisper
!pip install scikit-learn
!pip install openai-whisper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sqlite3
import pandas as pd
import zipfile
import io
import re
import nltk
import torch
import random
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import chromadb
from chromadb.utils import embedding_functions
import whisper
from nltk.tokenize import word_tokenize


In [4]:
# Download NLTK stopwords
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
# Step 1: Extract Subtitle Data from Database (Sample 30% of Data)
def extract_subtitles(db_path, sample_fraction=0.3):
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query("SELECT * FROM zipfiles", conn)
    conn.close()

    def decode_method(binary_data):
        try:
            with io.BytesIO(binary_data) as f:
                with zipfile.ZipFile(f, 'r') as zip_file:
                    subtitle_content = zip_file.read(zip_file.namelist()[0])
            return subtitle_content.decode('latin-1')
        except Exception as e:
            return ""

    df['file_content'] = df['content'].apply(decode_method)
    df = df.sample(n=int(len(df) * sample_fraction), random_state=42)  # Select 30% sample
    return df[['num', 'name', 'file_content']]


In [6]:
# Step 2: Text Cleaning and Preprocessing
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    words = word_tokenize(text)
    text = ' '.join([word for word in words if word not in stop_words])  # Remove stopwords
    return text



In [7]:
# Step 3: Chunking Text to Prevent Information Loss
def chunk_text(text, chunk_size=500, overlap=50):
    words = word_tokenize(text)
    chunks = []
    start = 0
    while start < len(words):
        end = start + chunk_size
        chunk = words[start:end]
        chunks.append(" ".join(chunk))
        start += chunk_size - overlap  # Apply overlap to avoid cutting context
    return chunks



In [8]:
# Step 4: Converting Audio to Text using Whisper
def audio_to_text(audio_path):
    try:
        model = whisper.load_model("base")
        result = model.transcribe(audio_path)
        return result["text"].strip()
    except Exception as e:
        print(f"Error in audio transcription: {e}")
        return ""



In [9]:
# TF-IDF Keyword-Based Search
def compute_tfidf(subtitles_df):
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(subtitles_df['file_content'])
    return vectorizer, tfidf_matrix



In [10]:
# chunking, embedding, storing in chromadb
def store_embeddings_in_chromadb(subtitles_df):
    print("Initializing ChromaDB...")
    client = chromadb.PersistentClient(path="./chroma_subtitle_db")
    collection = client.get_or_create_collection(name="subtitle_embeddings")

    print("Loading embedding model...")
    model = SentenceTransformer("all-MiniLM-L6-v2", device='cuda' if torch.cuda.is_available() else 'cpu')

    print("Cleaning text...")
    subtitles_df['cleaned_content'] = subtitles_df['file_content'].apply(clean_text)

    batch_size = 100
    print(f"Processing {len(subtitles_df)} subtitles in batches of {batch_size}...")

    for start in range(0, len(subtitles_df), batch_size):
        print(f"Processing batch {start} to {start + batch_size}...")
        end = start + batch_size
        batch = subtitles_df.iloc[start:end]

        all_chunks, all_ids, all_metadatas = [], [], []

        for num, name, content in zip(batch['num'], batch['name'], batch['cleaned_content']):
            chunks = chunk_text(content)
            all_chunks.extend(chunks)
            all_ids.extend([f"{num}_{i}" for i in range(len(chunks))])
            all_metadatas.extend([{"name": name, "content": chunk} for chunk in chunks])

        print(f"Encoding {len(all_chunks)} chunks...")
        with torch.no_grad():
            embeddings = model.encode(all_chunks, batch_size=32, show_progress_bar=True).tolist()

        print("Adding embeddings to ChromaDB...")
        collection.add(
            ids=all_ids,
            embeddings=embeddings,
            metadatas=all_metadatas
        )

    print("Embedding storage complete.")
    return collection



In [11]:
# function to search subtitles
def search_subtitles(audio_path, collection):
    model = SentenceTransformer("all-MiniLM-L6-v2", device='cuda' if torch.cuda.is_available() else 'cpu')
    query_text = audio_to_text(audio_path)
    query_embedding = model.encode([query_text]).tolist()
    results = collection.query(query_embeddings=query_embedding, n_results=5)

    # Formatting search results to display meaningful snippets
    formatted_results = []
    for i, (ids_list, metadatas_list, distances_list) in enumerate(zip(results['ids'], results['metadatas'], results['distances'])):
        for j in range(len(ids_list)):
            snippet = metadatas_list[j]['content'][:200] + "..." if len(metadatas_list[j]['content']) > 200 else metadatas_list[j]['content']
            formatted_results.append(f"Result {len(formatted_results)+1}: {metadatas_list[j]['name']}\nSnippet: {snippet}\nScore: {distances_list[j]:.4f}\n")
            if len(formatted_results) >= 5:
                break
        if len(formatted_results) >= 5:
            break

    return "\n".join(formatted_results)

In [12]:
db_path = "/content/drive/My Drive/eng_subtitles_database.db"

In [15]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [13]:
subtitles_df = extract_subtitles(db_path)

In [16]:
collection = store_embeddings_in_chromadb(subtitles_df)

Initializing ChromaDB...
Loading embedding model...
Cleaning text...
Processing 24749 subtitles in batches of 100...
Processing batch 0 to 100...
Encoding 675 chunks...


Batches:   0%|          | 0/22 [00:00<?, ?it/s]

Adding embeddings to ChromaDB...
Processing batch 100 to 200...
Encoding 615 chunks...


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

Adding embeddings to ChromaDB...
Processing batch 200 to 300...
Encoding 608 chunks...


...

Adding embeddings to ChromaDB...
Processing batch 24600 to 24700...
Encoding 663 chunks...


Batches:   0%|          | 0/21 [00:00<?, ?it/s]

Adding embeddings to ChromaDB...
Processing batch 24700 to 24800...
Encoding 322 chunks...


Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Adding embeddings to ChromaDB...
Embedding storage complete.


In [17]:
print(subtitles_df.head(10))

           num                                               name  \
17262  9251120                     maybe.this.time.(2014).eng.1cd   
7294   9211589  down.the.shore.s01.e10.and.justice.for.all.(19...   
47707  9380845  uncontrollably.fond.s01.e07.heartache.(2016).e...   
29914  9301436  screen.two.s13.e04.the.precious.blood.(1996).e...   
54266  9408707                          battlebots.(2015).eng.1cd   
76608  9497041  csi.crime.scene.investigation.s08.e16.two.and....   
72988  9480949                             royal.ashes.().eng.1cd   
82028  9519904                     return.to.seoul.(2022).eng.1cd   
81973  9519595  idris.elba.king.of.speed.s01.e02.episode.1.2.(...   
80364  9512716  tooth.pari.when.love.bites.s01.e08.episode.1.8...   

                                            file_content  \
17262  ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch...   
7294   1\r\n00:00:09,275 --> 00:00:11,876\r\n¶ Oh, I ...   
47707  1\r\n00:00:07,140 --> 00:00:14,220\r\n<i>Timin...   


In [21]:
audio_path = "/content/drive/My Drive/sample.mp3"

In [22]:
search_results = search_subtitles(audio_path, collection)


In [23]:
print(search_results)

Result 1: malazgirt.1071.(2022).eng.1cd
Snippet: fight sultan alparslan defend state religion make mistake made mistake listening son doesnt matter anymore may god bless son careful okay come come help come hurry quick find saddle come come come go ...
Score: 0.9661

Result 2: fauda.s04.e02.episode.4.2.(2022).eng.1cd
Snippet: muhammadi ithats accusation mei iyou know many muslims arrested dayi ihundreds day brussels around europei idue accusationi iwhat claim ive done promote peacei water drink gabi stubborn talk us talk u...
Score: 0.9770

Result 3: khorfakkan.(2020).eng.1cd
Snippet: fear alarmed time take us surprise thank god council decided evacuate town everyone fight abdullah abu ali know faith responsible evacuating town take much bring carry must get town safe portuguese ar...
Score: 0.9898

Result 4: kurulus.osman.s04.e19.117.bolum.(2023).eng.1cd
Snippet: stronger punish whoever responsible thats brother done words alone kumral abdal said strive practice never give brother com